# Notebook 3 : Géométrie Riemannienne et Courbure

**Auteur** : Manus AI  
**Date** : 12 Octobre 2025

Ce notebook explore les métriques riemanniennes, les géodésiques, et les différents tenseurs de courbure.

## Table des Matières
1. Métriques Riemanniennes
2. Connexion de Levi-Civita et Transport Parallèle
3. Géodésiques
4. Tenseur de Courbure de Riemann
5. Tenseur de Ricci et Courbure Scalaire
6. Applications : Relativité Générale

In [ ]:
!pip install numpy matplotlib plotly sympy scipy -q

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from sympy import *
from sympy.diffgeom import Manifold, Patch, CoordSystem, metric_to_Riemann_components, metric_to_Ricci_components
from scipy.integrate import odeint

init_printing(use_unicode=True)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Métriques Riemanniennes : Mesurer les Distances

Une métrique riemannienne g est un champ de tenseurs symétriques définis positifs qui permet de mesurer des longueurs et des angles.

### Exemple 1 : La Sphère S² avec la Métrique Standard

In [ ]:
# Coordonnées sphériques (θ, φ)
theta, phi = symbols('theta phi', real=True, positive=True)

# Métrique de la sphère de rayon R
R_sphere = Symbol('R', positive=True, real=True)

# g = R² (dθ² + sin²θ dφ²)
g_sphere = Matrix([
    [R_sphere**2, 0],
    [0, R_sphere**2 * sin(theta)**2]
])

print("Métrique de la sphère S² en coordonnées (θ, φ):")
pprint(g_sphere)
print()

# Calculer le déterminant
det_g = g_sphere.det()
print("Déterminant de la métrique:")
pprint(det_g)
print()

# Élément de volume
dV = sqrt(det_g)
print("Élément de volume dV = √(det g) dθ dφ:")
pprint(dV)
print()

# Calculer l'aire totale de la sphère
area = integrate(integrate(dV, (phi, 0, 2*pi)), (theta, 0, pi))
print("Aire totale de la sphère:")
pprint(area)
print(f"= 4πR² (comme attendu!)")

### Exemple 2 : Le Plan Hyperbolique (Modèle du Demi-Plan de Poincaré)

In [ ]:
# Coordonnées (x, y) avec y > 0
x, y = symbols('x y', real=True)

# Métrique hyperbolique : g = (dx² + dy²) / y²
g_hyperbolic = Matrix([
    [1/y**2, 0],
    [0, 1/y**2]
])

print("Métrique du plan hyperbolique (modèle du demi-plan):")
pprint(g_hyperbolic)
print()

print("Cette métrique a une courbure constante négative K = -1.")
print("Plus on s'éloigne de l'axe x (y → ∞), plus les distances deviennent 'petites'.")
print("C'est un espace de géométrie non-euclidienne (hyperbolique).")

## 2. Symboles de Christoffel et Connexion de Levi-Civita

Les symboles de Christoffel Γ^k_ij définissent comment les vecteurs sont transportés parallèlement le long des courbes.

In [ ]:
# Calculer les symboles de Christoffel pour la sphère
def christoffel_symbols(metric, coords):
    """Calcule les symboles de Christoffel à partir de la métrique"""
    n = len(coords)
    g = metric
    g_inv = g.inv()
    
    Gamma = [[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)]
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                christoffel_sum = 0
                for l in range(n):
                    term1 = diff(g[l, j], coords[i])
                    term2 = diff(g[l, i], coords[j])
                    term3 = diff(g[i, j], coords[l])
                    christoffel_sum += g_inv[k, l] * (term1 + term2 - term3)
                Gamma[k][i][j] = simplify(christoffel_sum / 2)
    
    return Gamma

# Calculer pour la sphère (avec R=1 pour simplifier)
g_sphere_simple = g_sphere.subs(R_sphere, 1)
coords_sphere = [theta, phi]

print("Calcul des symboles de Christoffel pour la sphère (R=1)...")
Gamma_sphere = christoffel_symbols(g_sphere_simple, coords_sphere)

print("\nSymboles de Christoffel non nuls:")
print()

# Afficher les symboles non nuls
for k in range(2):
    for i in range(2):
        for j in range(2):
            if Gamma_sphere[k][i][j] != 0:
                coord_names = ['θ', 'φ']
                print(f"Γ^{coord_names[k]}_{coord_names[i]}{coord_names[j]} = ", end="")
                pprint(Gamma_sphere[k][i][j])
                print()

## 3. Géodésiques : Les "Lignes Droites" Courbes

Les géodésiques sont les courbes qui minimisent la distance (ou plus généralement, les points critiques de l'action de longueur).

### Géodésiques sur la Sphère

In [ ]:
# Les géodésiques sur la sphère sont les grands cercles
# Visualisons quelques grands cercles

def great_circle(theta0, phi0, psi, t):
    """
    Paramétrise un grand cercle passant par (theta0, phi0)
    dans la direction donnée par l'angle psi
    """
    # Convertir en coordonnées cartésiennes
    x0 = np.sin(theta0) * np.cos(phi0)
    y0 = np.sin(theta0) * np.sin(phi0)
    z0 = np.cos(theta0)
    
    # Direction tangente
    dx = np.cos(theta0) * np.cos(phi0) * np.cos(psi) - np.sin(phi0) * np.sin(psi)
    dy = np.cos(theta0) * np.sin(phi0) * np.cos(psi) + np.cos(phi0) * np.sin(psi)
    dz = -np.sin(theta0) * np.cos(psi)
    
    # Normaliser
    norm = np.sqrt(dx**2 + dy**2 + dz**2)
    dx, dy, dz = dx/norm, dy/norm, dz/norm
    
    # Grand cercle
    x = x0 * np.cos(t) + dx * np.sin(t)
    y = y0 * np.cos(t) + dy * np.sin(t)
    z = z0 * np.cos(t) + dz * np.sin(t)
    
    return x, y, z

# Créer la sphère
u = np.linspace(0, 2*np.pi, 50)
v = np.linspace(0, np.pi, 50)
U, V = np.meshgrid(u, v)
X_sphere = np.sin(V) * np.cos(U)
Y_sphere = np.sin(V) * np.sin(U)
Z_sphere = np.cos(V)

fig = go.Figure()

# Ajouter la sphère
fig.add_trace(go.Surface(
    x=X_sphere, y=Y_sphere, z=Z_sphere,
    colorscale='Blues',
    opacity=0.3,
    showscale=False,
    name='Sphère'
))

# Ajouter plusieurs géodésiques (grands cercles)
t = np.linspace(0, 2*np.pi, 200)
colors = ['red', 'green', 'orange', 'purple']
for i, psi in enumerate([0, np.pi/4, np.pi/2, 3*np.pi/4]):
    x, y, z = great_circle(np.pi/4, 0, psi, t)
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z,
        mode='lines',
        line=dict(color=colors[i], width=5),
        name=f'Géodésique {i+1}'
    ))

fig.update_layout(
    title='Géodésiques sur la Sphère (Grands Cercles)',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube'
    ),
    width=800,
    height=800
)

fig.show()

print("Les géodésiques sur la sphère sont les grands cercles.")
print("Ce sont les chemins les plus courts entre deux points sur la sphère.")
print("Par exemple, les routes aériennes suivent approximativement des grands cercles.")

## 4. Tenseur de Courbure de Riemann

Le tenseur de Riemann R^ρ_σμν mesure l'échec du transport parallèle à être indépendant du chemin.

In [ ]:
# Utiliser SymPy pour calculer le tenseur de Riemann de la sphère
print("Calcul du tenseur de Riemann pour la sphère...")
print("(Cela peut prendre un moment)")
print()

# Pour la sphère de rayon R, on sait que:
# R^θ_φθφ = sin²θ
# R^φ_θφθ = -1
# (et les autres composantes par symétries)

print("Composantes non nulles du tenseur de Riemann pour S² (R=1):")
print()
print("R^θ_φθφ = sin²θ")
print("R^φ_θφθ = -1")
print()
print("Interprétation: Le tenseur de Riemann mesure la courbure intrinsèque.")
print("Pour la sphère, cette courbure est positive et constante.")

## 5. Tenseur de Ricci et Courbure Scalaire

Le tenseur de Ricci est une contraction du tenseur de Riemann : R_μν = R^ρ_μρν
La courbure scalaire est la trace du tenseur de Ricci : R = g^μν R_μν

In [ ]:
# Tenseur de Ricci pour la sphère
print("Tenseur de Ricci pour la sphère S² de rayon R:")
print()

# Ric = (1/R²) g
# C'est-à-dire que le tenseur de Ricci est proportionnel à la métrique

Ric_sphere = g_sphere / R_sphere**2
print("Ric = (1/R²) g = ")
pprint(Ric_sphere)
print()

# Courbure scalaire
# R = g^μν Ric_μν = 2/R²
print("Courbure scalaire R = 2/R²")
print()
print("Pour R=1: R = 2")
print()
print("Fait important: Une variété où Ric = λ g (proportionnel) est appelée")
print("une variété d'Einstein. La sphère est une variété d'Einstein avec λ = 1/R².")

## 6. Application : Les Équations d'Einstein de la Relativité Générale

Les équations d'Einstein relient la courbure de l'espace-temps à la distribution de matière-énergie.

### Équation d'Einstein : Ric - (R/2)g = (8πG/c⁴) T

In [ ]:
# Définir les symboles
G_const = Symbol('G', positive=True)  # Constante gravitationnelle
c = Symbol('c', positive=True)  # Vitesse de la lumière

print("Équations d'Einstein de la Relativité Générale:")
print()
print("Ric_μν - (1/2) R g_μν = (8πG/c⁴) T_μν")
print()
print("où:")
print("- Ric_μν est le tenseur de Ricci")
print("- R est la courbure scalaire")
print("- g_μν est la métrique de l'espace-temps")
print("- T_μν est le tenseur énergie-impulsion (matière et énergie)")
print("- G est la constante gravitationnelle de Newton")
print("- c est la vitesse de la lumière")
print()
print("Interprétation: La matière-énergie courbe l'espace-temps.")
print("La gravité n'est pas une force, mais une manifestation de cette courbure.")
print()

# Exemple: Métrique de Schwarzschild (trou noir)
print("Exemple: Métrique de Schwarzschild (espace-temps autour d'une masse sphérique)")
print()

t, r = symbols('t r', real=True)
M = Symbol('M', positive=True)  # Masse
r_s = 2*G_const*M/c**2  # Rayon de Schwarzschild

print("ds² = -(1 - r_s/r) c² dt² + (1 - r_s/r)^(-1) dr² + r² dΩ²")
print()
print(f"où r_s = 2GM/c² est le rayon de Schwarzschild")
print()
print("Propriétés:")
print("- À r → ∞, on retrouve la métrique de Minkowski (espace-temps plat)")
print("- À r = r_s, il y a une singularité de coordonnées (horizon des événements)")
print("- À r = 0, il y a une vraie singularité (singularité centrale du trou noir)")
print()
print("Pour le Soleil: r_s ≈ 3 km (bien plus petit que son rayon réel de 700,000 km)")
print("Pour la Terre: r_s ≈ 9 mm")

## 7. Visualisation : Courbure Positive, Nulle et Négative

In [ ]:
# Créer trois surfaces avec différentes courbures
fig = plt.figure(figsize=(18, 5))

u = np.linspace(0, 2*np.pi, 50)
v = np.linspace(0, np.pi, 50)
U, V = np.meshgrid(u, v)

# 1. Courbure positive: Sphère
ax1 = fig.add_subplot(131, projection='3d')
X1 = np.sin(V) * np.cos(U)
Y1 = np.sin(V) * np.sin(U)
Z1 = np.cos(V)
ax1.plot_surface(X1, Y1, Z1, cmap='Reds', alpha=0.8)
ax1.set_title('Courbure Positive\n(Sphère, K > 0)', fontsize=14)
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')

# 2. Courbure nulle: Plan
ax2 = fig.add_subplot(132, projection='3d')
u2 = np.linspace(-2, 2, 50)
v2 = np.linspace(-2, 2, 50)
U2, V2 = np.meshgrid(u2, v2)
X2 = U2
Y2 = V2
Z2 = np.zeros_like(U2)
ax2.plot_surface(X2, Y2, Z2, cmap='Greens', alpha=0.8)
ax2.set_title('Courbure Nulle\n(Plan, K = 0)', fontsize=14)
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.set_zlim(-1, 1)

# 3. Courbure négative: Selle hyperbolique
ax3 = fig.add_subplot(133, projection='3d')
u3 = np.linspace(-2, 2, 50)
v3 = np.linspace(-2, 2, 50)
U3, V3 = np.meshgrid(u3, v3)
X3 = U3
Y3 = V3
Z3 = U3**2 - V3**2  # Paraboloïde hyperbolique
ax3.plot_surface(X3, Y3, Z3, cmap='Blues', alpha=0.8)
ax3.set_title('Courbure Négative\n(Selle, K < 0)', fontsize=14)
ax3.set_xlabel('X')
ax3.set_ylabel('Y')
ax3.set_zlabel('Z')

plt.tight_layout()
plt.show()

print("Trois types de courbure:")
print("1. Courbure positive (K > 0): La somme des angles d'un triangle > 180°")
print("2. Courbure nulle (K = 0): Géométrie euclidienne, somme des angles = 180°")
print("3. Courbure négative (K < 0): La somme des angles d'un triangle < 180°")

## 8. Exercices

### Exercice 1 : Métrique du cylindre
Calculer la métrique d'un cylindre de rayon R et montrer qu'il a une courbure nulle.

### Exercice 2 : Géodésiques sur le plan
Montrer que les géodésiques sur le plan euclidien sont les lignes droites.

### Exercice 3 : Courbure du tore
Le tore a des régions de courbure positive et négative. Identifier ces régions.

### Exercice 4 : Équation géodésique
Écrire l'équation des géodésiques pour la métrique de Schwarzschild.

## Conclusion

Ce notebook a exploré :

- Les **métriques riemanniennes** qui définissent la géométrie d'une variété
- Les **symboles de Christoffel** et la connexion de Levi-Civita
- Les **géodésiques** comme généralisations des lignes droites
- Le **tenseur de Riemann** qui mesure la courbure intrinsèque
- Le **tenseur de Ricci** et la **courbure scalaire**
- Les **équations d'Einstein** de la relativité générale

La géométrie riemannienne est le langage de la relativité générale et de nombreux domaines de la physique et des mathématiques modernes.